# Reduce light frame using master files

## 모듈 설치 및 버전 확인

아래 셀을 실행하면 이 노트북을 실행하는데 필요한 모듈을 설치하고 파이썬 및 관련 모듈의 버전을 확인할 수 있습니다.

In [9]:
# Install a pip package in the current Jupyter kernel
import importlib, sys, subprocess
print(f"sys.executable: {sys.executable}")

packages = "numpy, pandas, matplotlib, astropy, photutils, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"**** {pkg} module is now installed.")
    else: 
        print(f"******** {pkg} module is already installed.")
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")


sys.executable: /home/guitar79/anaconda3/envs/astro_Python_env/bin/python
******** numpy module is already installed.
******** pandas module is already installed.
******** matplotlib module is already installed.
******** astropy module is already installed.
******** photutils module is already installed.
******** version_information module is already installed.
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-12-10 00:23:42 (KST = GMT+0900) 
0 Python     3.12.7 64bit [GCC 11.2.0]
1 IPython    8.27.0
2 OS         Linux 6.8.0 49 generic x86_64 with glibc2.39
3 numpy      1.26.4
4 pandas     2.2.3
5 matplotlib 3.8.4
6 astropy    7.0.0
7 photutils  1.12.0
8 version_information 1.0.4


### import modules

In [10]:
#%%
from glob import glob
from pathlib import Path
import os
import numpy as np
import astropy.units as u

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

import warnings
warnings.filterwarnings('ignore')

In [11]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C2-Asteroid"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C3-EXO"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    MASTERDIR = Path(BDFDIR[0]) / _astro_utilities.master_dir
    if not MASTERDIR.exists():
        os.makedirs("{}".format(str(MASTERDIR)))
        print("{} is created...".format(str(MASTERDIR)))
    print ("MASTERDIR: ", format(MASTERDIR))
except : 
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'BL'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

verbose = False  

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/-_CAL-BDF_-_2017-05_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-21_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-22_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-24_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-9237_LIGHT_-_2017-05-18_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-9237_LIGHT_-_2017-05-25_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA60

In [ ]:
tryagain = False
trynightsky = True 

for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    if verbose == True :
        print(f"Starting: {str(DOINGDIR.parts[-1])}")
    
    sMASTERDIR = DOINGDIR / _astro_utilities.master_dir
    REDUCEDDIR = DOINGDIR / _astro_utilities.reduced_dir
    REDUC_nightsky = DOINGDIR / _astro_utilities.reduced_nightsky_dir

    if not sMASTERDIR.exists():
        os.makedirs(str(sMASTERDIR))
        if verbose == True :
            print("{} is created...".format(str(sMASTERDIR)))

    if not REDUCEDDIR.exists():
        os.makedirs(str(REDUCEDDIR))
        if verbose == True :
            print("{} is created...".format(str(REDUCEDDIR)))

    if not REDUC_nightsky.exists():
        os.makedirs("{}".format(str(REDUC_nightsky)))
        if verbose == True :
            print("{} is created...".format(str(REDUC_nightsky)))

    summary = yfu.make_summary(DOINGDIR/"*.fit*")
    if summary is not None :
        if verbose == True :
            #print(summary)
            print("len(summary):", len(summary))
            print("summary:", summary)
            #print(summary["file"][0])

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

        summary_master = yfu.make_summary(MASTERDIR/"*.fit*", 
                           verbose = False,
                           )
        if verbose == True :
            print("summary_master", summary_master)

        summary_master_dark = summary_master.loc[summary_master["IMAGETYP"] == "DARK"].copy()
        summary_master_dark.reset_index(inplace=True)
        if verbose == True :
            print("summary_master_dark", summary_master_dark)

        if 'EXPTIME' in summary_master_dark :
            check_exptimes = summary_master_dark['EXPTIME'].drop_duplicates()
            check_exptimes = check_exptimes.reset_index(drop=True)
            if verbose == True :
                print("check_exptimes", check_exptimes)

        for _, row in df_light.iterrows():

            fpath = Path(row["file"])
            ccd = yfu.load_ccd(fpath)
            filt = ccd.header["FILTER"]
            expt = ccd.header["EXPTIME"]

            idx = abs(summary_master_dark['EXPTIME'] - expt).idxmin()
            if verbose == True :
                print(idx)

            if (REDUCEDDIR / fpath.name).exists() and tryagain == False :
                if verbose == True :
                    print(f"reduction file already exists...\n{fpath.name}")
                pass
            else :
                try : 
                    if not (MASTERDIR / f"master_flat_{filt.upper()}_norm.fits").exists() :
                        if verbose == True :
                            print(f"{MASTERDIR}/master_flat_{filt.upper()}_norm.fits is not exists...")
                    else :
                        if (MASTERDIR / f"master_dark_{expt:.0f}sec.fits").exists() :
                            if verbose == True :
                                print(f"Reduce with master_dark_{expt:.0f}sec.fits ...")

                            red = yfu.ccdred(
                                ccd,
                                output=Path(f"{REDUCEDDIR/ fpath.name}"),
                                mdarkpath=str(MASTERDIR / "master_dark_{:.0f}sec.fits".format(expt)),
                                mflatpath=str(MASTERDIR / "master_flat_{}_norm.fits".format(filt.upper())),
                                # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                                overwrite=True,
                                )
                        else : 
                            if verbose == True :
                                print(f"Reduce with master_dark_{summary_master_dark['EXPTIME'][idx]:.0f}sec.fits is not exists...")
                            red = yfu.ccdred(
                                ccd,
                                output=Path(f"{REDUCEDDIR/ fpath.name}"),
                                mdarkpath=str(MASTERDIR / f"master_dark_{summary_master_dark['EXPTIME'][idx]:.0f}sec.fits"),
                                mflatpath=str(MASTERDIR / f"master_flat_{filt.upper()}_norm.fits"),
                                dark_scale = True,
                                exptime_dark = summary_master_dark['EXPTIME'][idx],
                                # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                                overwrite=True,
                                )
                        if verbose == True :
                            print (f"Reduce Reduce {fpath.name} +++...")

                except Exception as e: 
                    print("err:", e)
                    pass
                
    if trynightsky == True : 
        REDUCNSKYDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
        if not REDUCNSKYDIR.exists():
            os.makedirs("{}".format(str(REDUCNSKYDIR)))
            if verbose == True :
                print("{} is created...".format(str(REDUCNSKYDIR)))
    
        summary = yfu.make_summary(REDUCEDDIR /"*.fit*")
        if summary is not None :
            if verbose == True :
                print("len(summary):", len(summary))
                print("summary:", summary)
                #print(summary["file"][0])   

            df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
            df_light = df_light.reset_index(drop=True)

            if 'FILTER' in df_light :
                check_filters = df_light['FILTER'].drop_duplicates()
                check_filters = check_filters.reset_index(drop=True)
                if verbose == True :
                    print("check_filters", check_filters)

                for filt in check_filters:
                #for filt in ["V"]:
                    df_light_filt = df_light.loc[df_light["FILTER"] == filt].copy()
                    
                    if df_light_filt.empty:
                        if verbose == True :
                            print(f"The dataframe(df_light_filt) {filt} is empty")
                        pass
                    else:
                        if verbose == True :
                            print("len(df_light_filt):", len(df_light_filt))
                            print("df_light_filt:", df_light_filt)
                    
                        if (sMASTERDIR / f"nightskyflat-{filt}.fits").exists() and tryagain == False :
                            pass
                        else :      
                            File_Num = 80
                            if len(df_light_filt["file"]) > File_Num :
                                combine_lst = df_light_filt["file"].tolist()[:File_Num]
                            else : 
                                combine_lst = df_light_filt["file"].tolist()
                            if (sMASTERDIR / f"nightskyflat-{filt}_norm.fits").exists():
                                pass
                            else :
                                try : 
                                    ccd = yfu.imcombine(
                                                        combine_lst, 
                                                        combine="med",
                                                        scale="avg", 
                                                        scale_to_0th=False, #norm
                                                        reject="sc", 
                                                        sigma=2.5,
                                                        verbose=True,
                                                        memlimit = 2.e+11,
                                                        )
                                except :
                                    ccd = yfu.imcombine(
                                                        combine_lst, 
                                                        combine="med",
                                                        scale="avg", 
                                                        scale_to_0th=False, #norm
                                                        reject="sc", 
                                                        # sigma=2.5,
                                                        verbose=True,
                                                        memlimit = 2.e+11,
                                                        )
                                ccd.write(sMASTERDIR / f"nightskyflat-{filt}_norm.fits", overwrite=True)
                                if verbose == True :
                                    print (f"Create Create nightskyflat-{filt}_norm.fits +++...")

            for _, row in df_light.iterrows():
                fpath = Path(row["file"])
                ccd = yfu.load_ccd(REDUCEDDIR / fpath.name)
                filt = row["FILTER"]
                if (REDUCNSKYDIR / fpath.name).exists() and tryagain == False:
                    if verbose == True :
                        print(f"Nightsky reduction file already exists...\n{fpath.name}")
                    pass
                else :
                    try:    
                        ccd = yfu.ccdred(
                                        ccd, 
                                        mflatpath = sMASTERDIR / f"nightskyflat-{filt}_norm.fits",
                                        output = REDUCNSKYDIR / fpath.name
                                    )
                    except : 
                        # _Python_utilities.write_log(err_log_file, "FileNotFoundError") 
                        pass

All 98 keywords (guessed from /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_B_2017-05-19-12-05-03_70sec_RiLA600_STX-16803_-20c_2bin.fit) will be loaded.


All 100 keywords (guessed from /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin/reduced/NSV-7244_LIGHT_B_2017-05-19-12-05-03_70sec_RiLA600_STX-16803_-20c_2bin.fit) will be loaded.
